In [1]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.


from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output


app = Dash(__name__)
# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
excel_file = 'data.xlsx'
sheet_name = 'season A'

df = pd.read_excel(excel_file,sheet_name=sheet_name)
fig = px.bar(df, x="Districts" , y="cultivatedArea" ,title="Amount in Districts")

# grapghs for 2022 summary
crop_summary_A = pd.read_excel(excel_file,sheet_name='2022 summaryCrops (season A)')
inputs_summary = pd.read_excel(excel_file,sheet_name='2022 summaryInputs')
practice_summary = pd.read_excel(excel_file,sheet_name='2022 summaryPractices')

crop_summary_A_fig =  px.bar(crop_summary_A, x='crops', y=['cultivatedArea','harvestedArea'],
                             title="cultivated-summary (Ha)",barmode='group')
production_summary_A_fig =  px.line(crop_summary_A, x='crops', y='cropProduction',
                             title="crop-production-summary (MT)")

yield_summary_A_fig =  px.bar(crop_summary_A, x='crops', y='cropYield',
                             title="crop-yield-summary (Kg/ha)",barmode='group')

input_summary_A_fig =  px.bar(inputs_summary, x='inputs', y=['season A','season B','season C'],
                             title="inputs-summary (%)",barmode='group')

practices_summary_fig =  px.bar(practice_summary, x='practices', y=['season A','season B','season C'],
                             title="practices-summary (%)",barmode='group')

input_summary_A_fig.update_layout(
    font=dict(size=9, family='Arial', color='black'),
    title_x=0.5,  # Center the title
    title_font=dict(size=18)
)
practices_summary_fig.update_layout(
    font=dict(size=9, family='Arial', color='black'),
    title_x=0.5,  # Center the title
    title_font=dict(size=18)
)

# Get unique values from the "Districts" column for dropdown options
district_options = [{'label': district, 'value': district} for district in df['Districts'].unique()]
column_options = [{'label': column, 'value': column} for column in df.columns]

app.layout = html.Div(children=[
    html.H1(children='Executive Summary of Seasonal Agricultural (2022) ',style={'text-align':'center'}),

#     summary graph and selector
    html.Div(children=[
        html.H2(children='Summary of Seasonal Agricultural Survey main indicators '),
        html.Div([
        dcc.Dropdown(
        id='season-dropdown',
        options=[
            {'label': 'Season A', 'value': 'season A'},
            {'label': 'Season B', 'value': 'season B'},
            {'label': 'Season C', 'value': 'season C'},
        ],
        value='season A',  # Default selected value
        multi=False,
        style={'width': '100%','margin-bottom':'6px'}
    ),
        ]),
        
    html.Div(children=[
        dcc.Graph(
        id='crop-summary-A',
        figure=crop_summary_A_fig,
        style={'width': '60%', 'height': '50%'}
    ),
        dcc.Graph(
        id='yield-summary-A',
        figure=yield_summary_A_fig,
        style={'width': '60%', 'height': '50%'}
    ),

    ],style={'display':'flex','gap':'5px'}),
     html.Div(children=[
        dcc.Graph(
        id='production-summary-A',
        figure=production_summary_A_fig,
        style={'width': '60%', 'height': '50%'}
    ),
    ],style={'display':'flex','justify-content': 'center','margin-top':'6px'}),
    html.Div(children=[
        dcc.Graph(
        id='inputs-summary',
        figure=input_summary_A_fig,
        style={'width': '60%', 'height': '50%'}
    ),
        dcc.Graph(
        id='practices-summary',
        figure=practices_summary_fig,
        style={'width': '60%', 'height': '50%','background-color':'green'}
    ),
    ],style={'display':'flex','margin-top':'6px','gap':'6px'}),
  
    ],style={'background-color':'#fad673','padding':'10px','border-radius':'5px'}),
    
#     lower section
    html.Div([
        html.H2(children='Per district data'),
        html.Div([
        dcc.Dropdown(
        id='dropdown',
        options=column_options,
        value=df.columns[1],  # Default selected value (change index as needed)
        multi=False,
        style={'width': '100%'}
    ),
        dcc.Dropdown(
        id='season-two-dropdown',
        options=[
            {'label': 'Season A', 'value': 'season A'},
            {'label': 'Season B', 'value': 'season B'},
            {'label': 'Season C', 'value': 'season C'},
        ],
        value='season A',  # Default selected value
        multi=False,
        style={'width': '100%'}
    ),
    ], style={'display':'flex','gap':'5px','width':'40%','float':'right'}),
    html.Div([
        html.Div(children=[
         dcc.Graph(
        id='example-graph',
        figure=fig,
        style={'width': '60%', 'height': '50%','background-color':'green'}
    ),
        dcc.Graph(
        id='secondary-graph',
        style={'width': '50%', 'height': '50%'}
    ),
        dcc.Graph(
        id='comparison-graph',
        style={'width': '50%', 'height': '50%'}
    ),
        ],style={'display':'flex','gap':'6px'})

    ],style={'padding':'10px'})
    ],style={'width': '100%','background-color': '#60d1ca','display':'grid','margin-top':'20px','padding':'10px','border-radius':'5px'})

],style={'margin-bottom':'20px'})

# Define callback to update the graph based on dropdown selection
@app.callback(
    Output('example-graph', 'figure'),
    [Input('dropdown', 'value'),
     Input('season-two-dropdown', 'value')]
)
def update_bar (selected_y_axis,selected_season):
        df = pd.read_excel(excel_file, sheet_name=selected_season)
        # Create a bar chart for the selected district and y-axis
        bar_fig = px.bar(df, x="Districts", y=selected_y_axis,
                        labels={'value': f'{selected_y_axis}'})

        bar_fig.update_layout(
        title=f"{selected_y_axis} per District ({selected_season})<br><span style='color:orange;'>(Hover on a district's bar to see additional data)</span>",
        font=dict(size=9, family='Arial', color='black'),
        title_x=0.5,  # Center the title
        title_font=dict(size=18)
)
        return bar_fig

# Define callback to update the secondary graph based on hover data
@app.callback(
    [Output('secondary-graph', 'figure'),
     Output('comparison-graph', 'figure')],
    [Input('example-graph', 'hoverData'),
     Input('season-two-dropdown', 'value'),
     Input('dropdown', 'value'),
     Input('season-two-dropdown', 'value')
    
    ]
)
def update_secondary_graph(hover_data, selected_sheet,selected_y_axis,selected_season):
    # Check if hover data is available
    if hover_data is None:
        # Return empty figures if no hover data
        return px.line(), px.bar()
#           return None,None
    else:
    # Extract the selected district from hover data
            selected_district = hover_data['points'][0]['x']
    new_sheet_name = f'{selected_y_axis} ({selected_season})'

    # Read data from a different sheet (replace 'Sheet2' with the actual sheet name)
    secondary_df = pd.read_excel(excel_file, sheet_name=new_sheet_name)

    # Here, I'm using a simple scatter plot as an example
    secondary_fig = px.line(secondary_df, x='crops', y=selected_district,)

    secondary_fig.update_layout(
        title=f"Data for {selected_district} in {selected_y_axis}",
        font=dict(size=9, family='Arial', color='black'),
        title_x=0.5,  # Center the title
        title_font=dict(size=18)
    )

    data_a = pd.read_excel(excel_file, sheet_name='season A')
    data_b = pd.read_excel(excel_file, sheet_name='season B')
    data_c = pd.read_excel(excel_file, sheet_name='season C')

    # Filter the DataFrames based on the selected district
    filtered_a = data_a[data_a['Districts'] == selected_district]
    filtered_b = data_b[data_b['Districts'] == selected_district]
    filtered_c = data_c[data_c['Districts'] == selected_district]

    # Create a comparison bar graph
    comparison_fig = px.bar()
    comparison_fig.add_bar(x=filtered_a['Districts'], y=filtered_a[selected_y_axis], name='season A')
    comparison_fig.add_bar(x=filtered_b['Districts'], y=filtered_b[selected_y_axis], name='season B')
    comparison_fig.add_bar(x=filtered_c['Districts'], y=filtered_c[selected_y_axis], name='season C')

    comparison_fig.update_layout(title=f'Comparison for {selected_district} across Seasons',
                                 barmode='group')

    return secondary_fig, comparison_fig

@app.callback(
    [Output('crop-summary-A', 'figure'),
     Output('yield-summary-A', 'figure'),
     Output('production-summary-A', 'figure')],
    [Input('season-dropdown', 'value')]
)
def update_upper_graphs(selected_season):
    sheet = f"2022 summaryCrops ({selected_season})"
    secondary_df = pd.read_excel(excel_file, sheet_name=sheet)
    
    crop_summary_A_fig = px.bar(secondary_df, x='crops', y=['cultivatedArea', 'harvestedArea'],
                                title=f"Cultivated and Harvested Summary (Ha) ({selected_season})", barmode='group')

    production_summary_A_fig = px.line(secondary_df, x='crops', y='cropProduction',
                                       title=f"Crop Production Summary (MT) ({selected_season})")
    
    yield_summary_A_fig = px.bar(secondary_df, x='crops', y='cropYield',
                                 title=f"Crop Yield Summary (Kg/ha) ({selected_season})", barmode='group')
    
    return crop_summary_A_fig, yield_summary_A_fig, production_summary_A_fig

if __name__ == '__main__':
    app.run(debug=True)
